In [12]:
import numpy as np 
import random 
import re 
import tensorflow as tf 
from sklearn.model_selection import train_test_split 
from sklearn import preprocessing 

import scipy.io as sio
import os

from time import time
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

from PIL import Image

In [47]:
def RNA_to_feature(str_rna, str_structure): 
#此函数将输入的RNA及其二级结构转化为输入的特征值 
    # 接受两个字符串,第一个字符串是基因序列,第二个字符串是对应的二级结构 
#     print(str_rna,str_structure)
#     energy = re.findall(r'-{0,1}\d{1,2}.\d{1,2}', str_structure)[0] 
    
    #利用正则表达式提取自由能 
    rna, structure = '','' 
    
    #初始化RNA和二级结构的数组 
    for i in range(len(str_rna)): 
        #这个循环用来保证二级结构和编码同样长度，同时避免混入非RNA编码 
        if str_rna[i] is not 'A' and str_rna[i] is not 'C' and str_rna[i] is not 'G' and str_rna[i] is not 'U': 
            continue 
        rna += str_rna[i] 
        structure += str_structure[i] 
        structure2int = [0 for i in range(len(structure))] 
        for i in range(len(structure)): 
        #这个循环将二级结构的编码转换为0，1编码，方便三元组编码的运用 
            if structure[i] is '(' or ')': 
                structure2int[i]= 1 
            if structure[i] is '.': 
                structure2int[i]= 0 
    #接下来构造一个32维特征，用于储存三元组编码方式的特征值，以字典形式给出。并初始化为0。
    rna_dic = {'A000': 0, 'A001': 0, 'A010': 0, 'A011': 0, 'A100': 0,'A101': 0, 'A110': 0, 'A111': 0, 'C000': 0, 'C001': 0, 'C010': 0, 'C011': 0, 'C100': 0,'C101': 0, 'C110': 0, 'C111': 0, 'G000': 0, 'G001': 0, 'G010': 0, 'G011': 0, 'G100': 0,'G101': 0, 'G110': 0, 'G111': 0, 'U000': 0, 'U001': 0, 'U010': 0, 'U011': 0, 'U100': 0,'U101': 0, 'U110': 0, 'U111': 0} 
    #然后搭建一个特征数组，方便判断使用。 
    rna_index = ['A000', 'A001', 'A010', 'A011', 'A100','A101', 'A110', 'A111', 'C000', 'C001', 'C010', 'C011', 'C100','C101', 'C110', 'C111', 'G000', 'G001', 'G010', 'G011', 'G100','G101', 'G110', 'G111', 'U000', 'U001', 'U010', 'U011', 'U100','U101', 'U110', 'U111'] 
    for i in range(len(rna)-2): 
        str2all = rna[i]+ __builtins__.str(structure2int[i])+ __builtins__.str(structure2int[i+1])+ __builtins__.str(structure2int[i+2]) 
        rna_dic[str2all] += 1 
    formalInput = [0 for i in range(32)] 
    #初始化最终特征向量formalInput，作为最后返回的特征向量。
    count = 0; 
    for i in rna_index: 
        formalInput[count] = rna_dic[i] 
        count += 1
#     print(energy)
#     formalInput[32] = float(energy) 
    #print(formalInput)
    #最后，将formalInput作为最终特征向量返回。
    
    
    
    
    return formalInput
    
    

In [41]:
def input_form_x(data_path): 
#数据读取函数 
    with open(data_path) as data: 
        #打开文件 
        
        form_x = [] 
        fp = data.readlines() 
        #print(len(fp))
        for i in range(0, len(fp), 3): 
            #每次跳两个读取 
            #print(fp[i+ 2])
            form_x.append(RNA_to_feature(fp[i + 1], fp[i + 2])) 
            #将奇数行（RNA编码）和偶数行（二级结构）一同输入，一起进行转换
            
    return form_x

In [84]:
def stack_3bin_CA(X_3bin,X_CA):
    
    for j in range(len(X_3bin)):
        a=X_3bin[j]
        b=X_CA[j]
        X_3bin[j] = np.insert(b, 0, values=a, axis=0)
    #print(X_train_correct_3bin[0])  
    X_stack=X_3bin
    return X_stack


In [93]:
#读取图片获得训练集和测试集
def get_files(file_dir,lable):
    X = []
    Y = []
    a=0

    img=np.zeros(shape=(32,32), dtype=np.int16)
    for file in os.listdir(file_dir):
        
        img=np.array(Image.open(file_dir+file))

        X.insert(a,img)
        Y.append(lable)
        a +=1
        
    return X,Y
        #print(x)

    

correct_lable = [0,1]
wrong_lable =[1,0]

X_train_correct_CA, Y_train_correct = get_files('data\\train_correct_resize\\',correct_lable)

X_train_wrong_CA, Y_train_wrong = get_files('data\\train_wrong_resize\\',wrong_lable)

X_test_correct_CA, Y_test_correct = get_files('data\\test_correct_resize\\',correct_lable)

X_test_wrong_CA, Y_test_wrong = get_files('data\\test_wrong_resize\\',wrong_lable)







X_train_correct_3bin = input_form_x('data\\positive_training_dataset.txt')
X_train_wrong_3bin = input_form_x('data\\negative_training_dataset.txt')

X_test_correct_3bin = input_form_x('data\\positive_testing_dataset.txt')
X_test_wrong_3bin = input_form_x('data\\negative_testing_dataset.txt')



X_train_correct = stack_3bin_CA(X_train_correct_3bin,X_train_correct_CA)

X_train_wrong = stack_3bin_CA(X_train_wrong_3bin,X_train_wrong_CA)

X_test_correct = stack_3bin_CA(X_test_correct_3bin,X_test_correct_CA)

X_test_wrong = stack_3bin_CA(X_test_wrong_3bin,X_test_wrong_CA)

X1 = []
Y1 = [] 

X1.extend(X_train_correct)
X1.extend(X_train_wrong)
X1.extend(X_test_correct)
X1.extend(X_test_wrong)


Y1.extend(Y_train_correct)
Y1.extend(Y_train_wrong)
Y1.extend(Y_test_correct)
Y1.extend(Y_test_wrong)

dataset1={}
dataset1['data']=X1
dataset1['target'] = Y1



#将数据保存为mat格式
sio.savemat('data\\3bin_CA_dataset.mat',dataset1)















# 训练集和测试集的随机分割
x_train, x_test, y_train, y_test = train_test_split(X1 ,Y1, test_size=0.15,random_state=42) 
# x_train, x_test, y_train, y_test = train_test_split(X_train_correct + X_train_wrong + X_test_correct + X_test_wrong,
#                                                     Y_train_correct + Y_train_wrong + Y_test_correct + Y_test_wrong,
#                                                     test_size=0.15,random_state=42) 


1216

In [37]:
'''
测试：将1*32维的三元编码加入到32*32的元胞缩略图中
'''

a=X_test_correct[1]
b=XX[1]
c = np.insert(a, 0, values=b, axis=0)
print(a,'\n\n',b,'\n\n',c)

aaa=len(X_train_correct_CA)
bbb=len(X_train_correct_3bin)
print(aaa,'\t',bbb)

[[ 50  63  28 ...   7 235  65]
 [ 90 124  24 ...  67   8  71]
 [209 105  40 ...  10   0  57]
 ...
 [ 24   7  15 ... 183  10  60]
 [ 20   0  16 ... 114 251  58]
 [ 67 134  35 ... 191   4  59]] 

 [5, 1, 0, 1, 0, 1, 2, 7, 4, 0, 0, 4, 1, 0, 1, 7, 3, 1, 0, 3, 2, 1, 4, 4, 3, 2, 0, 1, 2, 2, 2, 6] 

 [[  5   1   0 ...   2   2   6]
 [ 50  63  28 ...   7 235  65]
 [ 90 124  24 ...  67   8  71]
 ...
 [ 24   7  15 ... 183  10  60]
 [ 20   0  16 ... 114 251  58]
 [ 67 134  35 ... 191   4  59]]


In [87]:
#初始化权值
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1,seed=1) #生成一个截断的正态分布
    return tf.Variable(initial)
    #return tf.Variable(tf.zeros(shape))

#初始化偏置
def bias_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1,seed=1)
    return tf.Variable(initial)
    #return tf.Variable(tf.zeros(shape))

#卷积层
def conv2d(x,W):
    #input tensor of shape [batch, in_height, in_width, in_channels]
    #filter / kernel tensor of shape [filter_height, filter_width, in_channels, out_channels]
    #strides[0]=strides[3]=1. strides[1]代表ｘ方向的步长，strids[2]代表ｙ方向的步长
    #padding: A string from "SAME", "VALID"
    return tf.nn.conv2d(x,W,strides=[1,1,1,1], padding='SAME')

#池化层
def max_pool(x):
    #ksize [1,x,y,1]
    return tf.nn.max_pool(x,ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')



def multilayer_perceptron(x_train, x_test, y_train, y_test): 

    x_test = np.reshape(x_test, (-1, 1056))
    x_train_size = np.size(x_train, 0)
    #定义两个placeholder
    x = tf.placeholder(tf.float32, [None, 1056]) 
    y = tf.placeholder(tf.float32, [None, 2])
    
    #改变x的格式转为４Ｄ的向量【batch, in_height, in_width, in_channels]
    x_image = tf.reshape(x,[-1, 32, 33, 1])

    #初始化第一个卷积层的权值和偏量
    W_conv1 = weight_variable([5,5,1,64])#5*5的采样窗口，３２个卷积核从4个平面抽取特征
    b_conv1 = bias_variable([64])#每一个卷积核一个偏置值

    #把x_image和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool(h_conv1)#进行max-pooling,12-by-40
    
#     #初始化第二个卷积层的权值和偏置
#     W_conv2 = weight_variable([3,3,16,32]) #5*5的采样窗口，64个卷积核从32个平面抽取特征
#     b_conv2 = bias_variable([32]) #每一个卷积核一个偏置值

#     #把H_pool1和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
#     h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
#     h_pool2 = max_pool(h_conv2)#6-by-5
    

    '''  
    23*20的图片第一次卷积后还是100*240第一次池化后变为50*120
    第二次卷积后为50*120,第二次池化后变为25*60
    进过上面操作后得到32张25*60的平面
    '''

    #初始化第一全链接层的权值
    W_fc1 = weight_variable([16*17*64,256]) #上一层有6*10*64个神经元,全连接层有1024个神经元
    b_fc1 = bias_variable([256])

    #把池化层2的输出扁平化为1维
    h_pool2_flat = tf.reshape(h_pool1,[-1,16*17*64])
    #求第一个全连接层的输出
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

    #keep_prob用了表示神经元的输出概率
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    #初始化第二个全连接层
    W_fc2 = weight_variable([256,2])
    b_fc2 = bias_variable([2])

    
    #计算输出
    #sigmoid
    prediction = tf.nn.sigmoid(tf.matmul(h_fc1_drop,W_fc2) + b_fc2)
    # 结果存放在一个布尔列表中
    correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
    
    #交叉熵代价函数
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=prediction))
    
    #使用AdamOptimizer进行优化
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    
    #求准确率
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))




    with tf.Session() as sess: 
        sess.run(tf.global_variables_initializer())
        #初始化参数 
        # Training cycle 
        for epoch in range(10000): 
            batch_index = random.randint(0, 1000) 
            #每一次从0-1000随机选出一个数，作为起始点 
            batch_data = x_train[batch_index:batch_index + 20] 
            batch_labels = y_train[batch_index:batch_index + 20] 
            #在上面所选的起始点开始，拿出32组数据进行一次训练 
            batch_data = np.reshape(batch_data, (-1, 1056))
            sess.run(optimizer,feed_dict={x: batch_data, y: batch_labels, keep_prob: 0.7})
            Accuracy,Loss = sess.run((accuracy,cost),feed_dict={x: batch_data, y: batch_labels, keep_prob: 0.7})
            
            print("Accuracy:", Accuracy)
            print("Loss:", Loss)
            print('')
            #进行训练 
#             if epoch%300 == 0: 
#                 print(epoch)
# #                 accuracy = sess.run(accuracy, feed_dict={x: x_test, y: y_test, keep_prob: 1.0})
# #                 print("Accuracy:", sess.run(accuracy, {x:x_test,y: y_test, keep_prob: 1.0}))
# #                 print("Loss:", sess.run(cost, {x:x_test,y: y_test, keep_prob: 1.0}))
#                 Accuracy,Loss = sess.run((accuracy,cost) ,{x:x_test,y: y_test, keep_prob: 1.0})
                
#                 print("Accuracy:", Accuracy)
#                 print("Loss:", Loss)

                #每隔300次 输出对应的次数以及测试集所测试的准确度 
        # Test model 
        
        print("Total Accuracy:", accuracy.eval({x:x_test, y:y_test, keep_prob: 1.0})) 
        #最后，当训练结束时，输出最终的准确率。

In [88]:
#学习率设置
learning_rate = 1e-4

multilayer_perceptron(x_train, x_test, y_train, y_test)

Accuracy: 0.5
Loss: 0.7652159

Accuracy: 0.55
Loss: 0.6842102

Accuracy: 0.4
Loss: 0.7842101

Accuracy: 0.5
Loss: 0.8342101

Accuracy: 0.3
Loss: 0.8221987

Accuracy: 0.55
Loss: 0.79622155

Accuracy: 0.8
Loss: 0.54622155

Accuracy: 0.45
Loss: 0.7892387

Accuracy: 0.5
Loss: 0.83923876

Accuracy: 0.4
Loss: 0.78834516

Accuracy: 0.5
Loss: 0.7888371

Accuracy: 0.65
Loss: 0.7012502

Accuracy: 0.4
Loss: 0.8892387

Accuracy: 0.6
Loss: 0.7132616

Accuracy: 0.5
Loss: 0.82024443

Accuracy: 0.45
Loss: 0.8702445

Accuracy: 0.65
Loss: 0.6632616

Accuracy: 0.35
Loss: 0.9632616

Accuracy: 0.6
Loss: 0.736277

Accuracy: 0.35
Loss: 0.9322559

Accuracy: 0.55
Loss: 0.78225595

Accuracy: 0.55
Loss: 0.751248

Accuracy: 0.35
Loss: 1.0272272

Accuracy: 0.45
Loss: 0.8822559

Accuracy: 0.4
Loss: 0.9012502

Accuracy: 0.8
Loss: 0.56856936

Accuracy: 0.55
Loss: 0.7702444

Accuracy: 0.5
Loss: 0.80125016

Accuracy: 0.45
Loss: 0.7462219

Accuracy: 0.5
Loss: 0.83225596

Accuracy: 0.3
Loss: 0.9822548

Accuracy: 0.45
Los

Accuracy: 0.4
Loss: 0.9132616

Accuracy: 0.35
Loss: 0.9632616

Accuracy: 0.5
Loss: 0.8132616

Accuracy: 0.7
Loss: 0.6132616

Accuracy: 0.35
Loss: 0.9632616

Accuracy: 0.55
Loss: 0.7632616

Accuracy: 0.45
Loss: 0.8632616

Accuracy: 0.7
Loss: 0.6132616

Accuracy: 0.6
Loss: 0.7132616

Accuracy: 0.5
Loss: 0.8132616

Accuracy: 0.4
Loss: 0.9132616

Accuracy: 0.3
Loss: 1.0132616

Accuracy: 0.5
Loss: 0.8132616

Accuracy: 0.4
Loss: 0.9132616

Accuracy: 0.45
Loss: 0.8632616

Accuracy: 0.35
Loss: 0.9632616

Accuracy: 0.35
Loss: 0.9632616

Accuracy: 0.5
Loss: 0.8132616

Accuracy: 0.45
Loss: 0.8632616

Accuracy: 0.35
Loss: 0.9632616

Accuracy: 0.6
Loss: 0.7132616

Accuracy: 0.65
Loss: 0.66326165

Accuracy: 0.45
Loss: 0.8632616

Accuracy: 0.5
Loss: 0.8132616

Accuracy: 0.5
Loss: 0.8132616

Accuracy: 0.45
Loss: 0.8632616

Accuracy: 0.65
Loss: 0.6632616

Accuracy: 0.45
Loss: 0.8632616

Accuracy: 0.4
Loss: 0.9132616

Accuracy: 0.6
Loss: 0.71326166

Accuracy: 0.6
Loss: 0.7132616

Accuracy: 0.4
Loss: 0.9

Accuracy: 0.55
Loss: 0.7632616

Accuracy: 0.3
Loss: 1.0132616

Accuracy: 0.4
Loss: 0.9132616

Accuracy: 0.55
Loss: 0.7632616

Accuracy: 0.45
Loss: 0.8632616

Accuracy: 0.45
Loss: 0.8632616

Accuracy: 0.45
Loss: 0.8632616

Accuracy: 0.65
Loss: 0.6632616

Accuracy: 0.35
Loss: 0.9632616

Accuracy: 0.65
Loss: 0.66326165

Accuracy: 0.35
Loss: 0.9632616

Accuracy: 0.55
Loss: 0.7632616

Accuracy: 0.4
Loss: 0.9132616

Accuracy: 0.5
Loss: 0.8132616

Accuracy: 0.6
Loss: 0.7132616

Accuracy: 0.75
Loss: 0.5632616

Accuracy: 0.6
Loss: 0.7132616

Accuracy: 0.4
Loss: 0.9132616

Accuracy: 0.45
Loss: 0.8632616

Accuracy: 0.35
Loss: 0.9632616

Accuracy: 0.5
Loss: 0.8132616

Accuracy: 0.4
Loss: 0.9132616

Accuracy: 0.8
Loss: 0.5132617

Accuracy: 0.4
Loss: 0.9132616

Accuracy: 0.5
Loss: 0.8132616

Accuracy: 0.55
Loss: 0.7632616

Accuracy: 0.45
Loss: 0.8632616

Accuracy: 0.35
Loss: 0.9632616

Accuracy: 0.55
Loss: 0.7632616

Accuracy: 0.4
Loss: 0.9132616

Accuracy: 0.6
Loss: 0.7132616

Accuracy: 0.6
Loss: 0

Accuracy: 0.35
Loss: 0.9632616

Accuracy: 0.4
Loss: 0.9132616

Accuracy: 0.55
Loss: 0.7632616

Accuracy: 0.35
Loss: 0.9632616

Accuracy: 0.35
Loss: 0.9632616

Accuracy: 0.35
Loss: 0.9632616

Accuracy: 0.5
Loss: 0.8132616

Accuracy: 0.6
Loss: 0.7132616

Accuracy: 0.55
Loss: 0.7632616

Accuracy: 0.55
Loss: 0.7632616

Accuracy: 0.4
Loss: 0.9132616

Accuracy: 0.65
Loss: 0.6632616

Accuracy: 0.35
Loss: 0.9632616

Accuracy: 0.4
Loss: 0.9132616

Accuracy: 0.5
Loss: 0.8132616

Accuracy: 0.65
Loss: 0.66326165

Accuracy: 0.6
Loss: 0.71326166

Accuracy: 0.3
Loss: 1.0132616

Accuracy: 0.7
Loss: 0.6132616

Accuracy: 0.5
Loss: 0.8132616

Accuracy: 0.55
Loss: 0.7632617

Accuracy: 0.55
Loss: 0.7632616

Accuracy: 0.45
Loss: 0.8632616

Accuracy: 0.35
Loss: 0.9632616

Accuracy: 0.5
Loss: 0.8132616

Accuracy: 0.65
Loss: 0.6632616

Accuracy: 0.6
Loss: 0.7132616

Accuracy: 0.55
Loss: 0.7632616

Accuracy: 0.5
Loss: 0.8132616

Accuracy: 0.55
Loss: 0.7632616

Accuracy: 0.8
Loss: 0.5132616

Accuracy: 0.35
Loss:

Accuracy: 0.65
Loss: 0.6632616

Accuracy: 0.35
Loss: 0.9632616

Accuracy: 0.4
Loss: 0.9132616

Accuracy: 0.7
Loss: 0.6132616

Accuracy: 0.55
Loss: 0.7632616

Accuracy: 0.45
Loss: 0.8632616

Accuracy: 0.45
Loss: 0.8632616

Accuracy: 0.7
Loss: 0.6132616

Accuracy: 0.7
Loss: 0.6132616

Accuracy: 0.3
Loss: 1.0132616

Accuracy: 0.5
Loss: 0.8132616

Accuracy: 0.6
Loss: 0.7132616

Accuracy: 0.55
Loss: 0.7632617

Accuracy: 0.3
Loss: 1.0132616

Accuracy: 0.45
Loss: 0.8632616

Accuracy: 0.35
Loss: 0.9632616

Accuracy: 0.6
Loss: 0.7132616

Accuracy: 0.4
Loss: 0.9132616

Accuracy: 0.8
Loss: 0.5132617

Accuracy: 0.6
Loss: 0.7132616

Accuracy: 0.25
Loss: 1.0632616

Accuracy: 0.45
Loss: 0.8632616

Accuracy: 0.4
Loss: 0.9132616

Accuracy: 0.45
Loss: 0.8632616

Accuracy: 0.6
Loss: 0.7132616

Accuracy: 0.65
Loss: 0.6632616

Accuracy: 0.5
Loss: 0.8132616

Accuracy: 0.55
Loss: 0.7632616

Accuracy: 0.6
Loss: 0.7132616

Accuracy: 0.35
Loss: 0.9632616

Accuracy: 0.4
Loss: 0.9132616

Accuracy: 0.7
Loss: 0.613

...

Accuracy: 0.3
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.3
Loss: 0.6931472

Accuracy: 0.65
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.3
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931

Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.65
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.65
Loss: 0.6931472

Accuracy: 0.35
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.65
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.35
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.65
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.3
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.35
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.65
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accurac

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.3
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.65
Loss: 0.6931472

Accuracy: 0.25
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.35
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.25
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.35
Loss: 0.6931472

Accuracy: 0.35
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.65
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.25
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.35
Loss

Accuracy: 0.65
Loss: 0.6931472

Accuracy: 0.65
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.3
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.35
Loss: 0.6931472

Accuracy: 0.7
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.8
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.693147

KeyboardInterrupt: 

In [89]:
x = tf.placeholder(tf.float32, [None, 1056]) 
y = tf.placeholder(tf.float32, [None, 2])
learning_rate = 1e-4
x_test = np.reshape(x_test, (-1, 1056))

def multilayer_perceptron(x, weights, biases): 
    # 隐藏层采用ReLU激活函数 
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1']) 
    layer_1 = tf.nn.relu(layer_1) 
    # 采用ReLU激活函数 
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2']) 
    layer_2 = tf.nn.relu(layer_2) 
    
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3']) 
    layer_3 = tf.nn.relu(layer_3) 
    
    layer_4 = tf.add(tf.matmul(layer_3, weights['h4']), biases['b4']) 
    layer_4 = tf.nn.relu(layer_4) 
    # 输出层采用线性激活函数 
    out_layer = tf.matmul(layer_4, weights['out']) + biases['out'] 
    return out_layer 
    #返回输出层的值

    
weights = { 'h1': tf.Variable(tf.random_normal([1056, 512])), 
           'h2': tf.Variable(tf.random_normal([512, 256])), 
           'h3': tf.Variable(tf.random_normal([256, 128])),
           'h4': tf.Variable(tf.random_normal([128, 32])),
           'out': tf.Variable(tf.random_normal([32, 2])) } 

biases = { 'b1': tf.Variable(tf.random_normal([512])), 
          'b2': tf.Variable(tf.random_normal([256])), 
          'b3': tf.Variable(tf.random_normal([128])),
          'b4': tf.Variable(tf.random_normal([32])), 
          'out': tf.Variable(tf.random_normal([2])) } 

pred = multilayer_perceptron(x, weights, biases) 

logits = tf.nn.softmax(pred) 

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y)) 
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)   



correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(y,1))
#得出通过正确个数除以总数得出准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))


    
# 训练集和测试集的随机分割

init = tf.global_variables_initializer()
with tf.Session() as sess: 
    sess.run(init) 
    #初始化参数 
    # Training cycle 
    for epoch in range(100000): 
        batch_index = random.randint(0, 1000) 
        #每一次从0-1000随机选出一个数，作为起始点 
        batch_data = x_train[batch_index:batch_index + 32] 
        batch_labels = y_train[batch_index:batch_index + 32] 
        #在上面所选的起始点开始，拿出32组数据进行一次训练 
        batch_data = np.reshape(batch_data, (-1, 1056))
        sess.run([optimizer, cost], feed_dict={x: batch_data, y: batch_labels})
        #进行训练 
        if epoch%500 == 0: 
            print(epoch)
            #accuracy = sess.run(accuracy, feed_dict={x: x_test, y: y_test, keep_prob: 1.0})
            print("Accuracy:", sess.run(accuracy, {x:x_test,y: y_test}))
            print("Loss:", sess.run(cost, {x:x_test,y: y_test}))
            
            #每隔300次 输出对应的次数以及测试集所测试的准确度 
    # Test model 
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1)) 
    # Calculate accuracy 
    #accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float")) 
    print("Total Accuracy:", accuracy.eval({x:x_test, y:y_test})) 
    #最后，当训练结束时，输出最终的准确率。

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

0
Accuracy: 0.5300546
Loss: 4079605.2
500
Accuracy: 0.52459013
Loss: 1901340.4
1000
Accuracy: 0.5464481
Loss: 1654303.1
1500
Accuracy: 0.5464481
Loss: 1618562.6
2000
Accuracy: 0.5409836
Loss: 1559078.6
2500
Accuracy: 0.5409836
Loss: 1542322.9
3000
Accuracy: 0.5300546
Loss: 1548680.0
3500
Accuracy: 0.5355191
Loss: 1544320.5
4000
Accuracy: 0.5355191
Loss: 1548520.8
4500
Accuracy: 0.5300546
Loss: 1564810.0
5000
Accuracy: 0.5355191
Loss: 1560242.8
5500
Accuracy: 0.5464481
Loss: 1525973.4


KeyboardInterrupt: 